## [DACON] 진동데이터 활용 충돌체 탐지 AI 경진대회
## 1Gb (팀명) (Team name)
## 2020년 월 일 (제출날짜) (Submission date)

## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D, Input, Conv1D, Multiply, Add, Dropout, GlobalMaxPooling2D, GlobalAvgPool2D
import keras.backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from keras import regularizers
from keras.models import load_model
from keras import models
import pandas as pd
import tensorflow as tf
import sys

from sklearn.model_selection import KFold
from scipy.signal import find_peaks

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
seed_everything(62)

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [ ]:
# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        train_input_mean = train[col].mean()
        train_input_sigma = train[col].std()
        train[col] = (train[col] - train_input_mean) / train_input_sigma
        test[col] = (test[col] - train_input_mean) / train_input_sigma
    
    return train, test

In [ ]:
DATA_PATH = './data/KAERI_dataset'

In [ ]:
def get_itg(signal_feat):
    itg = np.trapz(signal_feat.values, axis=0)
    return itg

def rolling_features(train, test):
    
    pre_train = train.copy()
    pre_test = test.copy()
    
        
    for df in [pre_train, pre_test]:
                
        for window in [50]:
            
            # roll backwards
            df['S1mean_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).mean())

            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).std())

            df['S1var_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).var())

            df['S1min_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).min())
            
            
            df['S1max_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).max())
            
            min_max = (df['S1'] - df['S1min_t' + str(window) + '_S1']) / (df['S1max_t' + str(window) + '_S1'] - df['S1min_t' + str(window) + '_S1'])
            df['norm_t' + str(window) + '_S1'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S1']) - np.ceil(df['S1min_t' + str(window) + '_S1']))
            min_max = (df['S2'] - df['S1min_t' + str(window) + '_S2']) / (df['S1max_t' + str(window) + '_S2'] - df['S1min_t' + str(window) + '_S2'])
            df['norm_t' + str(window) + '_S2'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S2']) - np.ceil(df['S1min_t' + str(window) + '_S2']))
            min_max = (df['S3'] - df['S1min_t' + str(window) + '_S3']) / (df['S1max_t' + str(window) + '_S3'] - df['S1min_t' + str(window) + '_S3'])
            df['norm_t' + str(window) + '_S3'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S3']) - np.ceil(df['S1min_t' + str(window) + '_S3']))
            min_max = (df['S4'] - df['S1min_t' + str(window) + '_S4']) / (df['S1max_t' + str(window) + '_S4'] - df['S1min_t' + str(window) + '_S4'])
            df['norm_t' + str(window) + '_S4'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S4']) - np.ceil(df['S1min_t' + str(window) + '_S4']))

    del train, test, min_max
    
    
    return pre_train.fillna(0), pre_test.fillna(0)
# get lead and lags features
def lag_with_pct_change(df, windows):
    
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        for window in windows:
            df[col + '_pos_' + str(window)] = df.groupby('id')[col].shift(window).fillna(0)
            df[col + '_shift_neg_' + str(window)] = df.groupby('id')[col].shift(-1 * window).fillna(0)
    
    return df

def get_cusum(df):
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        df[col + '_cum_sum'] = df.groupby('id')[col].cumsum().fillna(0)
    return df

def com_gradient(signal_faet, step=1):
    g = signal_faet
    for _ in range(step):
        g = np.gradient(g)
    return g

def get_gradient(df, step=1):
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        df[col + '_g_' + f'{step}'] = np.concatenate(df.groupby('id')['S1'].apply(lambda x : com_gradient(x.values, 1)).values)

    return df

def com_peak(signal_feat):
    '''
    signal_feat -> B, S, F
    B : number of data
    S : sequncer size
    F : number of feature
    '''
    peaks = np.zeros_like(signal_feat)
    
    # find peak
    peak_1, _ = find_peaks(signal_feat)
    peak_2, _ = find_peaks(-signal_feat)
    peaks[peak_1] = 1
    peaks[peak_2] = -1
    
    return peaks

def get_peak(df):
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        df[col + '_peak'] = np.concatenate(df.groupby('id')[col].apply(lambda x : com_peak(x)).values)
    
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df):

    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2])
    # cumerate sum
    df = get_cusum(df)
    # get gradient
#     df = get_gradient(df, 1)
#     df = get_gradient(df, 2)
    # get peak
    df = get_peak(df)
    
    return df
def create_feature():
    train = pd.read_csv(DATA_PATH + '/train_features.csv')
    test = pd.read_csv(DATA_PATH + '/test_features.csv')
    
    train, test = normalize(train, test)
    train, test = rolling_features(train, test)
    
    train = run_feat_engineering(train)
    test = run_feat_engineering(test)
    feture_dir = './new_feature'

    if not os.path.exists(feture_dir):
        os.mkdir(feture_dir)
    train.to_csv(feture_dir+'/new_feat_train_V6.csv',index=False)
    test.to_csv(feture_dir+'/new_feat_test_V6.csv',index=False)

In [ ]:
create_feature()

In [ ]:
# Read original data
X_data = pd.read_csv(DATA_PATH+'/train_features.csv')
X_data_test = pd.read_csv(DATA_PATH+'/test_features.csv')


# Normalize train test
X_data, X_data_test = normalize(X_data, X_data_test)

# Drop unnecessary colunm
X_data = X_data.drop(columns=['id'], axis=1).values
X_data_test = X_data_test.drop(columns=['id'], axis=1).values

# Resize
X_data = X_data.reshape((2800,375,5,1))
X_data_test = X_data_test.reshape((700,375,5,1))

# Check shape
print(X_data.shape)
print(X_data_test.shape)

# Read Target value
Y_data = np.loadtxt(DATA_PATH+'/train_target.csv',skiprows=1,delimiter=',')
Y_data = Y_data[:,1:]
print(Y_data.shape)

In [ ]:
# New Feature directory
FEAT_PATH = './new_feature'

# X_new_feat_trn = np.loadtxt(FEAT_PATH+'/new_feat_train_V2.csv',skiprows=1,delimiter=',')
X_new_feat_trn = pd.read_csv(FEAT_PATH+'/new_feat_train_V6.csv')
X_new_feat_trn = X_new_feat_trn.drop(columns=['id'], axis=1).values
X_new_feat_trn = X_new_feat_trn.reshape((2800,375,50,1))
print(X_new_feat_trn.shape)


# # X_new_feat_test = np.loadtxt(FEAT_PATH+'/new_feat_test_V2.csv',skiprows=1,delimiter=',')
X_new_feat_test = pd.read_csv(FEAT_PATH+'/new_feat_test_V6.csv')
X_new_feat_test = X_new_feat_test.drop(columns=['id'], axis=1).values
X_new_feat_test = X_new_feat_test.reshape((700,375,50,1))
print(X_new_feat_test.shape)

## 3. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [ ]:
weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])

def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))


def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04

def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)

In [ ]:
def kaeri_metric(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: KAERI metric
    '''
    
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)


### E1과 E2는 아래에 정의됨 ###

def E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)


def E2(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    
    
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

def E2M(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2], np.array(y_pred)[:,2]
    
    
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

def E2V(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,-1], np.array(y_pred)[:,-1]
    
    
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

In [ ]:
def set_model(train_target):  # 0:x,y, 1:m, 2:v
    
    activation = 'elu'
    padding = 'valid'
    model = Sequential()
    nf = do_nf
    fs = (3,1)
    
    if train_target == 2:
        if do_slice:
            model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(200,5,1)))
        else:
            model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(375,5,1)))
    else:
        if do_slice:
            model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(200,50,1)))
        else:
            model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(375,50,1)))
            
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*2,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*4,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*8,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*16,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*32,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    if do_gap:
        if train_target == 0:
            model.add(Flatten())
        else:
            model.add(GlobalAvgPool2D())
    else:
        model.add(Flatten())
    model.add(Dense(128, activation ='elu'))
    model.add(Dense(64, activation ='elu'))
    model.add(Dense(32, activation ='elu'))
    model.add(Dense(16, activation ='elu'))
    model.add(Dense(4))
    
    optimizer = keras.optimizers.Adam()

    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    
    if train_target==0:
        model.compile(loss=my_loss_E1,
                  optimizer=optimizer,
                 )
    else:
        model.compile(loss=my_loss_E2,
                  optimizer=optimizer,
                 )
       

    return model

## 4. 모델 학습 및 검증
## Model Tuning & Evaluation

In [ ]:
def train(model,X,Y, split, train_target):
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
    trn_id, val_id = split[0], split[1]

    best_save = ModelCheckpoint('./model/'+'best_m.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, min_lr=0.00001)
    
    callbacks = list()
    callbacks.append(best_save)
    callbacks.append(reduce_lr)
    
    trn_X, trn_Y = X[trn_id], Y[trn_id]
    val_X, val_Y = X[val_id], Y[val_id]
    
    if train_target == 2:
        
        history = model.fit(X, Y,
                      epochs=500,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(val_X, val_Y),
                      verbose = 2,
                      callbacks=callbacks)
    else:
        history = model.fit(X, Y,
                      epochs=300,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(val_X, val_Y),
                      verbose = 2,
                      callbacks=callbacks)
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.axvline(x=np.argmin(history.history['val_loss']), color='b', linewidth=1)
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    
    
    return model

In [ ]:
def load_best_model(train_target, train_X, train_Y, train_new_feat_X):

    if train_target == 0:
        model = load_model('./model/best_m.hdf5' , custom_objects={'my_loss_E1': my_loss, })
    else:
        model = load_model('./model/best_m.hdf5' , custom_objects={'my_loss_E2': my_loss, })
  
    if train_target == 2:
        pred = model.predict(train_X)
    else:
        pred = model.predict(train_new_feat_X)
    
    i=0
    print('정답(original):', train_Y[i])
    print('예측값(original):', pred[i])
    e1_score = E1(train_Y, pred)
    e2_score = E2(train_Y, pred)
    e2m_score = E2M(train_Y, pred)
    e2v_score = E2V(train_Y, pred)
    
    print('E1 : ', e1_score)
    print('E2 : ', e2_score)
    print('E2M : ', e2m_score)
    print('E2V : ', e2v_score)    
    
    
    return model, [e1_score, e2_score, e2m_score, e2v_score]

In [ ]:
def do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test):
    submit = pd.read_csv('../data/KAERI_dataset/sample_submission.csv')
    
    if do_slice:
        d_X_data = X_data[:, :200]
        d_X_data_test = X_data_test[:, :200]

        d_X_new_feat_trn = X_new_feat_trn[:, :200]
        d_X_new_feat_test = X_new_feat_test[:, :200]
    else:
        d_X_data = X_data
        d_X_data_test = X_data_test

        d_X_new_feat_trn = X_new_feat_trn
        d_X_new_feat_test = X_new_feat_test

    if do_flip:
        d_X_data = np.flip(d_X_data, axis=1)
        d_X_data_test = np.flip(d_X_data_test, axis=1)

        d_X_new_feat_trn = np.flip(d_X_new_feat_trn, axis=1)
        d_X_new_feat_test = np.flip(d_X_new_feat_test, axis=1)
        
        
    kfold = KFold(n_splits=n_fold, shuffle=False, random_state=42)
    splits = [x for x in kfold.split(X_data, Y_data)]
    
    score_list = list()
    for train_target in range(3):

        for split in splits:
            model = set_model(train_target)

            if train_target == 2:
                train(model, d_X_data, Y_data, split, train_target)
            else:
                train(model, d_X_new_feat_trn, Y_data, split, train_target)

            best_model, scores = load_best_model(train_target, d_X_data, Y_data, d_X_new_feat_trn)

            if train_target == 2:
                pred_data_test = best_model.predict(d_X_data_test)
            else:
                pred_data_test = best_model.predict(d_X_new_feat_test)

            score_list.append(scores)

            if train_target == 0: # x,y 학습
                submit.iloc[:,1] += pred_data_test[:,0] / n_fold
                submit.iloc[:,2] += pred_data_test[:,1] / n_fold

            elif train_target == 1: # m 학습
                submit.iloc[:,3] += pred_data_test[:,2] / n_fold

            elif train_target == 2: # v 학습
                submit.iloc[:,4] += pred_data_test[:,3] / n_fold

    submit.to_csv(pred_name, index=False)
    print(score_list)

In [ ]:
def pred_naming(do_nf, do_gap, do_flip, do_slice):
    pred_name = './cnn'
    pred_name += f'_nf{do_nf}'
    if do_gap:
        pred_name += '_gap'
    if do_flip:
        pred_name += '_flip'
    if not do_slice:
        pred_name += '_375'

    pred_name += '_pred.csv'
    
    print(pred_name)
    
    return pred_name

In [ ]:
# case 1
do_slice=True
do_gap=False
do_nf=32
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 2
do_slice=True
do_gap=False
do_nf=32
do_flip=True
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 3
do_slice=True
do_gap=False
do_nf=16
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 4
do_slice=True
do_gap=False
do_nf=16
do_flip=True
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 5
do_slice=True
do_gap=True
do_nf=16
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 6
do_slice=True
do_gap=True
do_nf=16
do_flip=True
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 7
do_slice=True
do_gap=True
do_nf=32
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 8
do_slice=True
do_gap=True
do_nf=32
do_flip=True
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 9
do_slice=False
do_gap=False
do_nf=16
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)

In [ ]:
# case 10
do_slice=False
do_gap=True
do_nf=16
do_flip=False
n_fold = 5

pred_name = pred_naming(do_nf, do_gap, do_flip, do_slice)
do_train(do_slice, do_flip, pred_name, X_new_feat_trn, X_new_feat_test)